In [2]:
import cv2
import mediapipe as mp


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands


In [14]:
def getFrames(wnum, seq): # 차례대로 단어 번호, 그 안의 영상 번호
    cap = cv2.VideoCapture(f"../dataset/{str(wnum).zfill(2)}/{str(seq).zfill(2)}_{str(wnum).zfill(2)}.MP4")
    max_len = 150

    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)

    frames = []

    ftime = 0
    fgap = 1
    if int(fps) > 15:
        fgap = 2

    while(cap.isOpened()):
        ret, image = cap.read()
        if not ret:
            break

        if ftime % fgap == 0:
            image = cv2.normalize(src=image, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            frames.append(image)
        ftime += 1

    while(len(frames) < max_len):
        frames.append(0)

    return frames

In [15]:
def facemesh_video(wnum, seq):
    
    facemesh_payload = []
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        print(getFrames(wnum, seq)[38])
    
        for idx, frame in enumerate(getFrames(wnum, seq)):
            # Convert the BGR image to RGB before processing.
            
            if type(frame) == int: # 프레임이 존재하지 않으면
                if idx == 0: # 첫 프레임부터 안 보이면 0 넣는다.
                    facemesh_payload.append(0)
                    continue
                
                print(len(facemesh_payload))
                print(f'{idx}번째 프레임에는 프레임이 감지되지 않음')
                facemesh_payload.append(facemesh_payload[idx-1])
                continue
            
            results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            if not results.multi_face_landmarks: # 혹은 프레임에서 얼굴 감지가 안 되면 (영상이 짧음)
                if idx == 0: # 첫 프레임부터 안 보이면 0 넣는다.
                    facemesh_payload.append(0)
                    continue
                
                print(f'{idx}번째 프레임에는 얼굴이 감지되지 않음')
                facemesh_payload.append(facemesh_payload[idx-1])
                continue
            
            frame_landmarks = []
            
            for lm in results.multi_face_landmarks[0].landmark:
                x = lm.x
                y = lm.y
                z = lm.z
                
                frame_landmarks.append([x, y, z])
            
            #print(f'{idx}번째 프레임 :', frame)
            #print(f'{idx}번째 프레임, 이 프레임에서 잡히는 랜드마크 갯수 :',len(a[0].landmark))
            
#             print(frame_landmarks)
            facemesh_payload.append(frame_landmarks)
    
        return facemesh_payload
        


In [16]:
facemesh_pay = facemesh_video(0, 0)
print(len(facemesh_pay))

[[[ 55  78  65]
  [ 55  78  65]
  [ 53  78  65]
  ...
  [180 192 218]
  [175 185 211]
  [177 187 213]]

 [[ 55  78  65]
  [ 55  78  65]
  [ 53  78  65]
  ...
  [181 193 219]
  [176 186 212]
  [178 188 214]]

 [[ 51  78  65]
  [ 51  78  65]
  [ 51  78  65]
  ...
  [182 194 220]
  [182 191 219]
  [182 191 219]]

 ...

 [[ 24  23  40]
  [ 24  23  40]
  [ 26  25  42]
  ...
  [ 14   4  17]
  [ 16   3  17]
  [ 16   3  17]]

 [[ 26  26  40]
  [ 26  26  40]
  [ 29  26  43]
  ...
  [ 10   0  13]
  [ 12   0  13]
  [ 12   0  13]]

 [[ 26  26  40]
  [ 26  26  40]
  [ 29  26  43]
  ...
  [ 10   0  13]
  [ 11   0  12]
  [ 11   0  12]]]
1번째 프레임에는 얼굴이 감지되지 않음
2번째 프레임에는 얼굴이 감지되지 않음
3번째 프레임에는 얼굴이 감지되지 않음
46번째 프레임에는 얼굴이 감지되지 않음
57
57번째 프레임에는 프레임이 감지되지 않음
58
58번째 프레임에는 프레임이 감지되지 않음
59
59번째 프레임에는 프레임이 감지되지 않음
60
60번째 프레임에는 프레임이 감지되지 않음
61
61번째 프레임에는 프레임이 감지되지 않음
62
62번째 프레임에는 프레임이 감지되지 않음
63
63번째 프레임에는 프레임이 감지되지 않음
64
64번째 프레임에는 프레임이 감지되지 않음
65
65번째 프레임에는 프레임이 감지되지 않음
66
66번째 프레임에는 프레임이 감지되지 않음
67
67번째 프레임

In [53]:
def points_to_displacement(face_points, face_count, hand_points, hand_count): #face_count: 얼굴 특징점 갯수, hand_count: 한 손 특징점 갯수
    displacement_payload = []
    
    # face mash
        
    nz = False
    for idx, frame in enumerate(face_points):
        if type(frame) == int:
            displacement_payload.append({"face":[[0,0,0] for _ in range(face_count)]})
        elif not nz:
            nz = True
            displacement_payload.append({"face":[[0,0,0] for _ in range(face_count)]})
        else:
            displacements = []
            for i in range(face_count):
                # print(f"{frame[i][0]}-{points[idx][i][0]}")
                displacements.append({"face":[frame[i][0]-face_points[idx-1][i][0],
                                              frame[i][1]-face_points[idx-1][i][1],
                                              frame[i][2]-face_points[idx-1][i][2]]})
            displacement_payload.append(displacements)
            
            
    # hand pose estimation
    lr = ["left", "right"]
    nz[2] = [False, False]
    for idx, frame in enumerate(hand_points):
        hand_displacements = {}
        for i, hand in enumerate(frame):
            if type(frame) == int:
                hand_displacements[lr[i]] = [[0,0,0] for _ in range(hand_count)]
            elif not nz[i]:
                hand_displacements[lr[i]] = [[0,0,0] for _ in range(hand_count)]
                nz[i] = True
            else:
                hand_displacements[lr[i]] = [hand[i][0]-hand_points[idx-1][i][0],
                                             hand[i][1]-hand_points[idx-1][i][1],
                                             hand[i][2]-hand_points[idx-1][i][2]]
            displacement_payload[idx]["hands"] = hand_displacements
            
    return displacement_payload

In [50]:
disp = points_to_displacement(facemesh_pay, 0)

In [52]:
print(disp[0])

{'face': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], 

In [ ]:
facemesh_pay = facemesh_video(1, 1)
print(len(facemesh_pay[0]))